In [15]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from pprint import pprint
from enum import Enum
from IPython.display import HTML, display
import pandas as pd
from io import StringIO
import os

In [16]:
# CONST
USER_ID = os.environ["UNIPA_USER_ID"]
PLAIN_PASSWORD = os.environ["UNIPA_PLAIN_PASSWORD"]

In [17]:
class LoginState(Enum):
    Success = 0
    FAILED = 1
    MAINTENANCE = 2
    TIMEOUT = 3

# TOOD: without login_url ;(
def getLoginState(soup, login_url) -> LoginState:
   if 'メンテナンス' in str(soup.body):
      return LoginState.MAINTENANCE
   if 'タイムアウト' in str(soup.body):
      return LoginState.TIMEOUT
   if soup.find(id="form1").attrs.get("action") == login_url:
      return LoginState.FAILED
   return LoginState.Success

In [18]:
session = requests.Session()

In [19]:
BASE_URL = 'https://unipa.itp.kindai.ac.jp'
url = BASE_URL + '/up/faces/login/Com00501A.jsp'

res = session.get(url)
soup = BeautifulSoup(res.text, "html.parser")

In [20]:
# id=form1 action="/up/faces/login/Com00504A.jsp"
login_url = soup.find(id="form1").attrs.get("action")
token = soup.find(id="com.sun.faces.VIEW").attrs.get("value")

print("LOGIN_URL:", BASE_URL + login_url)
print("TOKEN    :", token)

LOGIN_URL: https://unipa.itp.kindai.ac.jp/up/faces/login/Com00501A.jsp
TOKEN    : _id17686:_id17687


In [21]:
params = urllib.parse.urlencode({
    'form1:htmlUserId': USER_ID, # 学籍番号
    'form1:htmlPassword': PLAIN_PASSWORD, # パスワード
    'form1:login.x': '50', # ﾋｰ
    'form1:login.y': '10', # ﾋｰ
    "form1:htmlNextFuncId":	"",
    "form1:htmlHiddenSsoFlg": "",
    "form1:htmlHiddenUserId": "",
    "form1:htmlHiddenPassword":	"",
    "form1:htmlHiddenUnipaSso":	"",
    "com.sun.faces.VIEW": token,
    "form1": "form1",
})

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    # "Cookie": ";".join(cookies),
}

r = session.post(url=BASE_URL + login_url, data=params, headers=headers)
print(r)
soup = BeautifulSoup(r.text, "html.parser")


<Response [200]>


In [23]:
if (state := getLoginState(soup, login_url)) != LoginState.Success:
    raise Exception("Login Failed", state)

# 雑名前取得
soup.find(id="header").find("span").text.split("\xa0")[0].replace("\u3000", " ")

# 最終ログイン時間
soup.find(id="header").find("span").text.split("\xa0")[-1]

'2024/04/14 01:42'

In [24]:
def decodeText(text):
    return text.replace("\u3000", " ").replace(">", "").replace("<", "").replace(" ", "").replace("\xa0", "")

# listから\nを抹殺する
def removelistEnter(array):
    return list(filter(lambda x: x != "\n", array))

In [25]:
links = {}

for content in soup.find(id="menubox").contents:
    if content == "\n": continue

    id = content.attrs.get("id")
    menu_title = content.find(class_="menuhead").text

    item = []
    # サブメニューの中にサブメニューがある場合は一旦無視する。
    for a in content.find(class_="submenu").find_all("a"):
        onclick = a.attrs.get("onclick")
        if onclick is None:
            continue
        item.append({
            "text": decodeText(a.text),
            "onclick": onclick,
        })
    links[id] = item

pprint(links)

{'menu1': [{'onclick': 'clickMenuItem(101,0);', 'text': '学籍情報照会'},
           {'onclick': 'clickMenuItem(109,0);', 'text': '住所等変更申請'},
           {'onclick': 'clickMenuItem(110,0);', 'text': '各種申請登録'},
           {'onclick': 'clickSiteMenuItem(111,0);', 'text': '奨学金申込'},
           {'onclick': 'clickSiteMenuItem(10501,0);', 'text': 'KudosKnowledge'},
           {'onclick': 'clickSiteMenuItem(10502,0);', 'text': 'KindaiMail'},
           {'onclick': 'clickSiteMenuItem(10505,0);', 'text': '学費振込'}],
 'menu2': [{'onclick': 'clickMenuItem(201,0);', 'text': '学生時間割表'},
           {'onclick': 'clickMenuItem(202,0);', 'text': '授業時間割表'},
           {'onclick': 'clickMenuItem(203,0);', 'text': '教員時間割表'},
           {'onclick': 'clickMenuItem(211,0);', 'text': '学生出欠状況確認'},
           {'onclick': 'clickSiteMenuItem(215,0);', 'text': 'シラバス照会'}],
 'menu3': [{'onclick': 'clickMenuItem(311,0);', 'text': '履修登録'},
           {'onclick': 'clickMenuItem(331,0);', 'text': '成績照会'}],
 'menu4': [{'onclick': 'c

In [26]:
# めんどくさいので一旦決め打ち
# 'menu2': [{'onclick': 'clickMenuItem(201,0);', 'text': '学生時間割表'},
menuNo,funcRowId = 201, 0
token = soup.find(id="com.sun.faces.VIEW").attrs.get("value")

params = urllib.parse.urlencode({	
  "header:form1:htmlMenuItemButton": "実行",
  "header:form1:hiddenMenuNo": str(menuNo),
  "header:form1:hiddenFuncRowId":str(funcRowId),
  "com.sun.faces.VIEW": token,
  "header:form1": "header:form1",
})

r = session.post(url=BASE_URL + login_url, data=params, headers=headers)
print(r)

<Response [200]>


In [27]:
soup = BeautifulSoup(r.text, "html.parser")

if (state := getLoginState(soup, login_url)) != LoginState.Success:
    raise Exception("Login Failed", state)

#  	開講年度 学期 表示形式 form
soup.find_all("table")[4]

# 開講年度, 学期
print(soup.find_all("table")[5].text.split())

# 時間割テーブル
table = soup.find_all("table")[6]
# display(HTML(str(table)))
contents = removelistEnter([content.text for content in table.find("tr").contents])

csv = ",".join(contents)

items = []
for cell in table.find("tbody").find_all("tr"):
    item = removelistEnter([decodeText(tr.text)[0:6] or "　"*6 for tr in cell])
    csv += "\n" + ",".join(item)
    items.append(item)

# transpose
timetable = list(zip(*items))

for i, item in enumerate(timetable):
    if i == 0:
        print(f"| 　  |　　", " 　　　| 　　".join(item), "　　　|")
        print( "-" * 84)
        continue
    print(f"| {i}限 |", " | ".join(item), "|")

['2024年度', '前期']
| 　  |　　 月 　　　| 　　火 　　　| 　　水 　　　| 　　木 　　　| 　　金 　　　| 　　土 　　　|
------------------------------------------------------------------------------------
| 1限 | 　　　　　　 | 　　　　　　 | 　　　　　　 | 　　　　　　 | 基礎微分積分 | 　　　　　　 |
| 2限 | 基礎線形代数 | 近大ゼミ【石 | 　　　　　　 | 情報処理実習 | 　　　　　　 | 　　　　　　 |
| 3限 | コンピュータ | プログラミン | 　　　　　　 | 　　　　　　 | 　　　　　　 | 　　　　　　 |
| 4限 | 英語総合１（ | プログラミン | 英語総合１（ | 　　　　　　 | 　　　　　　 | 　　　　　　 |
| 5限 | 　　　　　　 | 　　　　　　 | 韓国語総合１ | オーラルイン | 　　　　　　 | 　　　　　　 |
| 6限 | 　　　　　　 | 　　　　　　 | 　　　　　　 | 　　　　　　 | 　　　　　　 | 　　　　　　 |
| 7限 | 　　　　　　 | 　　　　　　 | 　　　　　　 | 　　　　　　 | 　　　　　　 | 　　　　　　 |


In [28]:
pd.read_csv(StringIO(csv)).T

,0,1,2,3,4,5
Unnamed: 0,月,火,水,木,金,土
1時限,,,,,基礎微分積分,
2時限,基礎線形代数,近大ゼミ【石,,情報処理実習,,
3時限,コンピュータ,プログラミン,,,,
4時限,英語総合１（,プログラミン,英語総合１（,,,
5時限,,,韓国語総合１,オーラルイン,,
6時限,,,,,,
7時限,,,,,,
